# Using HuggingFace

The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [23]:
#Loading document loaders
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

### Chunking the documents

In [4]:
#Read the pdfs from the folder
loader = PyPDFDirectoryLoader("./files")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

final_documents = text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'files\\file_1.pdf', 'page': 0})

In [5]:
len(final_documents)

316

### Embeddings

In [7]:
#Embeddings using Hugging Face
hgf_embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en-v1.5", #sentence-transformers/all-MiniLM-l6-v2
                         model_kwargs = {'device':'cpu'},
                         encode_kwargs={'normalize_embeddings':True}
                         )

c:\Users\ADMIN\Documents\Data Analytics\Machine Learning\Getting_started_with_Langchain\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\ADMIN\Documents\Data Analytics\Machine Learning\Getting_started_with_Langchain\venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#li

In [8]:
import  numpy as np
print(np.array(hgf_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(hgf_embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.46568793e-02 -1.19099347e-02 -3.37892920e-02  2.94559766e-02
  5.19159846e-02  5.73839881e-02 -4.10017557e-02  2.74268072e-02
 -1.05128214e-01 -1.58055872e-02  7.94858634e-02  5.64318411e-02
 -1.31765464e-02 -3.41543816e-02  5.81603311e-03  4.72547524e-02
 -1.30746849e-02  3.12988739e-03 -3.44225504e-02  3.08405999e-02
 -4.09086235e-02  3.52737531e-02 -2.43761446e-02 -4.35831696e-02
  2.41503362e-02  1.31986886e-02 -4.84451558e-03  1.92347281e-02
 -5.43912575e-02 -1.42735034e-01  5.15528955e-03  2.93115862e-02
 -5.60811013e-02 -8.53537768e-03  3.14141251e-02  2.76736021e-02
 -2.06188336e-02  8.24231356e-02  4.15425748e-02  5.79655282e-02
 -3.71586904e-02  6.26157969e-03 -2.41390169e-02 -5.61793894e-03
 -2.51715332e-02  5.04969666e-03 -2.52801143e-02 -2.91941711e-03
 -8.24047532e-03 -5.69604859e-02  2.30822749e-02 -5.54220239e-03
  5.11555821e-02  6.09937720e-02  6.49766549e-02 -5.38514070e-02
  2.19109785e-02 -2.54194289e-02 -4.49223146e-02  4.22458760e-02
  4.75252122e-02  7.23232

In [11]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],hgf_embeddings)

In [12]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [13]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023E0754C2B0> search_kwargs={'k': 3}


### Using Hugging Face LLM

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [26]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

'What is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured’s medical expenses in exchange for a premium. The insured agrees to pay the premium and to comply with the terms of the contract.\n\nWhat are the benefits of health insurance?\n\nThe benefits of health insurance include:\n\n1. Protection against financial ruin in the event of a serious illness or injury.\n\n2. Access to quality medical'

In [ ]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

In [18]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [19]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [27]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [29]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [30]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

comparison of ACS and CPS ASEC measures 
of health insurance coverage, refer to < www.
census.gov/topics/health/health-insurance/
guidance.html >.
9 Respondents may have more than one 
health insurance coverage type at the time 
of interview. As a result, adding the total 
number of people with private coverage and 
the total number with public coverage will 
sum to more than the total number with any 
coverage.• From 2021 to 2022, nine states 
reported increases in private 
coverage, while seven reported 
decreases (Appendix Table B-2). 
DIFFERENCES IN THE 
UNINSURED RATE BY STATE 
IN 2022
In 2022, uninsured rates at the 
time of interview ranged across 
states from a low of 2.4 percent 
in Massachusetts to a high of 16.6 
percent in Texas, compared to the 
national rate of 8.0 percent.10 Ten 
of the 15 states with uninsured 
10 The uninsured rates in the Distr